# Strands Agent Graph を用いたマルチエージェントシステムの構築
マルチエージェントシステムは、複数の専門AIエージェントが連携して複雑な問題を解決することで、協調的なコラボレーションを実現します。各エージェントはそれぞれ固有の機能と役割を持ち、明示的な通信経路によって接続されています。

このラボでは、Strands Agent Graph ツールを使用してマルチエージェントシステムを構築する方法を学習します。基本的な概念から高度な実装へと進み、さまざまなトポロジと実際のアプリケーションを探求します。

**学習目標:**
このノートブックを完了すると、以下のことができるようになります。
- エージェントグラフの3つのコアコンポーネント（ノード、エッジ、トポロジ）を理解する
- さまざまなトポロジパターン（スター、メッシュ、階層）を作成および管理する
- エージェントグラフに自然言語と構造化アプローチの両方を実装する
- 特定のエージェント間でターゲットメッセージを送信する
- マルチエージェントネットワークを監視および制御する
- 実際のシナリオに特化したエージェントシステムを設計する

## 前提条件

- Python 3.10以上
- Amazon BedrockでAnthropic Claude 3.7が有効になっているAWSアカウント
- Amazon Bedrockを使用する権限を持つIAMロール
- AIエージェントとPromptエンジニアリングに関する基本的な理解

## セットアップとインストール

始める前に、`strands-agents` と `strands-agents-tools` に必要なパッケージをインストールしましょう。

In [1]:
!pip install -r requirements.txt

### 必要なパッケージのインポート

次に、必要なパッケージをインポートします

In [2]:
import boto3
from strands import Agent
from strands_tools import agent_graph
import json

## エージェントグラフの構成要素について
エージェントグラフは、相互接続されたAIエージェントの構造化ネットワークであり、協調的なコラボレーションを通じて複雑な問題を解決するように設計されています。各エージェントは特定の機能を持つ特殊なノードを表し、エージェント間の接続は明確な通信経路を定義します。

構築を始める前に、エージェントグラフの3つの主要コンポーネントを理解しましょう。

### 1. ノード（エージェント）
各ノードは、以下の要素を持つAIエージェントを表します。
- **アイデンティティ**: グラフ内で一意の識別子
- **ロール**: 特殊な機能または目的
- **システムプロンプト**: エージェントの動作を定義する指示
- **ツール**: エージェントが利用できる機能
- **メッセージキュー**: 受信通信用のバッファ

### 2. エッジ（接続）
エッジは、以下の要素を持つ通信経路を定義します。
- **方向**: 一方向または双方向の情報フロー
- **関係**: エージェント間の関係（スーパーバイザー/ワーカー、ピアツーピア）
- **メッセージパッシング**: 情報を転送するためのメカニズム

### 3. トポロジパターン
ユースケースごとに異なるネットワーク構造：
- **スター**: 放射状に広がる中央コーディネータースペシャリストが中心となり、編集監修付きのコンテンツ作成やエスカレーションパスを備えたカスタマーサービスといった集中型ワークフローに最適です。
- **メッシュ**：すべてのエージェントが直接通信できる完全接続ネットワーク。共同作業による問題解決、議論、合意形成に最適です。
- **階層型**：親子関係を持つツリー構造。階層型処理、タスク委任によるプロジェクト管理、多階層レビュープロセスに最適です。

![トポロジ](images/topologies.png)

### 最初のエージェントグラフの作成 - スタートポロジ

まずは、編集上の監督を伴う集中型ワークフローに最適なスタートポロジから始めましょう。チームリーダーをコーディネーターエージェント、データアナリストとドメインエキスパートをスペシャリストエージェントとして構成するリサーチエージェントを構築します。アーキテクチャは以下のようになります。

<div style="text-align:left">
<img src="images/star_architecture.png" width="55%" />
</div>

In [ ]:
# ステップ 1: agent_graph機能を持つエージェントを初期化
coordinator_agent = Agent(tools=[agent_graph])

# ステップ 2: agent_graphツールを使用して構造化されたエージェントグラフを作成
result = coordinator_agent.tool.agent_graph(
    action="create",
    graph_id="research_team",
    topology={
        "type": "star",
        "nodes": [
            {
                "id": "coordinator",
                "role": "team_lead",
                "system_prompt": """あなたは、専門家を調整する研究チームのリーダーです。
                 簡単な分析を提供してください。フォローアップは必要ありません。"""
            },
            {
                "id": "data_analyst",
                "role": "analyst",
                "system_prompt": """あなたは統計分析を専門とするデータアナリストです。
                 簡単な分析を提供してください。フォローアップは必要ありません。"""
            },
            {
                "id": "domain_expert",
                "role": "expert",
                "system_prompt": """あなたは深い専門知識を持つドメインエキスパートです。
                 簡単な分析を提供してください。フォローアップは必要ありません。"""
            }
        ],
        "edges": [
            {"from": "coordinator", "to": "data_analyst"},
            {"from": "coordinator", "to": "domain_expert"},
            {"from": "data_analyst", "to": "coordinator"},
            {"from": "domain_expert", "to": "coordinator"}
        ]
    }
)

print("Research team created successfully!")
print(f"Result: {result}")

# ステップ 3: 新しく作成したネットワークのステータスを確認
status_result = coordinator_agent.tool.agent_graph(
    action="status",
    graph_id="research_team"
)

print("Network Status:")
print(status_result)

# ステップ 4: コーディネーターにタスクを送信
print("Sending task to the coordinator...")

message_result = coordinator_agent.tool.agent_graph(
    action="message",
    graph_id="research_team",
    message={
        "target": "coordinator",
        "content": """リモートワークが従業員の生産性に与える影響を分析します。簡単な分析を提供するため、フォローアップは不要です。"""
    }
)

print("Task sent to coordinator!")
print(f"Response: {message_result}")
print("============================================================")
print("============================================================")

# ステップ 5：専門家との直接コミュニケーション
print("Sending direct message to data analyst...")

analyst_result = coordinator_agent.tool.agent_graph(
    action="message",
    graph_id="research_team",
    message={
        "target": "data_analyst",
        "content": """リモートワーク分析で追跡すべき主要な生産性指標は何ですか？簡単な分析を提供すれば、フォローアップは必要ありません。"""
    }
)

print(f"Data Analyst Response: {analyst_result}")
print("============================================================")
print("============================================================")

# ステップ 6: アクティブなネットワークをすべて一覧表示する
print("Checking all active agent networks...")

list_result = coordinator_agent.tool.agent_graph(action="list")
print(f"Active Networks: {list_result}")
print("============================================================")
print("============================================================")
# ネットワークを管理する
coordinator_agent("コンテンツリサーチチームが終わったらシャットダウンする")

DEPRECATION WARNING: agent_graph will be removed in the next major release. Migration path: replace agent_graph calls with graph for equivalent functionality.


DEPRECATION WARNING: agent_graph will be removed in the next major release. Migration path: replace agent_graph calls with graph for equivalent functionality.


Research team created successfully!
Result: {'toolUseId': 'tooluse_agent_graph_394879916', 'status': 'success', 'content': [{'text': 'Graph research_team created and started'}]}


DEPRECATION WARNING: agent_graph will be removed in the next major release. Migration path: replace agent_graph calls with graph for equivalent functionality.


Network Status:
{'toolUseId': 'tooluse_agent_graph_595244434', 'status': 'success', 'content': [{'text': 'Graph research_team status retrieved.'}]}
Sending task to the coordinator...


...

DEPRECATION WARNING: agent_graph will be removed in the next major release. Migration path: replace agent_graph calls with graph for equivalent functionality.


Data Analyst Response: {'toolUseId': 'tooluse_agent_graph_864865676', 'status': 'success', 'content': [{'text': 'Message sent to node data_analyst'}]}
Checking all active agent networks...


DEPRECATION WARNING: use_llm will be removed in the next major release. Migration path: replace use_llm calls with use_agent for equivalent functionality.
DEPRECATION WARNING: use_llm will be removed in the next major release. Migration path: replace use_llm calls with use_agent for equivalent functionality.


Active Networks: {'toolUseId': 'tooluse_agent_graph_868012148', 'status': 'success', 'content': [{'text': 'Listed 1 active agent graphs.'}]}
了解しました。研究チーム「research_team」をシャットダウンします。
Tool #1: agent_graph


DEPRECATION WARNING: agent_graph will be removed in the next major release. Migration path: replace agent_graph calls with graph for equivalent functionality.


### メッシュトポロジの作成 - 協調的な問題解決

では、すべてのエージェントが直接通信できるメッシュトポロジを作成しましょう。これは、議論や合意形成に最適です。ファイナンシャルアドバイザー、テクニカルアーキテクト、市場調査担当者、リスクアナリストからなる意思決定委員会エージェントを作成します。アーキテクチャは以下のようになります。
<div style="text-align:left">
<img src="images/mesh_architecture.png" width="55%" />
</div>

In [ ]:
# ステップ 1: 共同意思決定のためのメッシュトポロジを作成する
print("🕸️ Creating mesh topology for collaborative analysis...")
mesh_agent = Agent(tools=[agent_graph])
mesh_result = mesh_agent.tool.agent_graph(
    action="create",
    graph_id="decision_committee",
    topology={
        "type": "mesh",
        "nodes": [
            {
                "id": "financial_advisor",
                "role": "finance_expert",
                "system_prompt": "あなたは、費用対効果分析、予算への影響、ROI計算に重点を置くファイナンシャルアドバイザーです。他の専門家と連携し、包括的な財務視点を構築しましょう。"
            },
            {
                "id": "technical_architect",
                "role": "tech_expert", 
                "system_prompt": "実現可能性、実装上の課題、そして技術的リスクを評価するテクニカルアーキテクトとして、他の専門家と連携し、技術的な実現可能性を確保します。"
            },
            {
                "id": "market_researcher",
                "role": "market_expert",
                "system_prompt": "あなたは市場調査員として、市場の状況、ユーザーニーズ、そして競合状況を分析します。他の専門家と協力し、市場機会を検証します。"
            },
            {
                "id": "risk_analyst",
                "role": "risk_expert",
                "system_prompt": "あなたは、潜在的なリスク、軽減戦略、コンプライアンス上の問題を特定するリスクアナリストです。他の専門家と連携し、包括的なリスク評価を実施します。"
            }
        ],
        "edges": [
            # フルメッシュ - 誰もが誰とでも話せる
            {"from": "financial_advisor", "to": "technical_architect"},
            {"from": "financial_advisor", "to": "market_researcher"},
            {"from": "financial_advisor", "to": "risk_analyst"},
            {"from": "technical_architect", "to": "financial_advisor"},
            {"from": "technical_architect", "to": "market_researcher"},
            {"from": "technical_architect", "to": "risk_analyst"},
            {"from": "market_researcher", "to": "financial_advisor"},
            {"from": "market_researcher", "to": "technical_architect"},
            {"from": "market_researcher", "to": "risk_analyst"},
            {"from": "risk_analyst", "to": "financial_advisor"},
            {"from": "risk_analyst", "to": "technical_architect"},
            {"from": "risk_analyst", "to": "market_researcher"}
        ]
    }
)

print("Mesh topology decision committee created!")
print(f"Result: {mesh_result}")
print("============================================================")
print("============================================================")

# ステップ 2：複雑な決定を委員会に提示
print("Presenting decision scenario to the committee...")

# まずファイナンシャルアドバイザーに送る
financial_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "financial_advisor",
        "content": "当社は、AIを活用した新しいカスタマーサービスプラットフォームの立ち上げを検討しています。初期投資額は200万ドルで、3年間のROIは150%と予測しています。財務評価はいかがでしょうか？"
    }
)

print(f"Financial Advisor: {financial_response}")
print("============================================================")
print("============================================================")

# ステップ 3: 技術的な視点を得る
technical_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "technical_architect",
        "content": "AI カスタマー サービス プラットフォームの提案に関して、主な技術的考慮事項、インフラストラクチャ要件、実装の課題は何ですか?"
    }
)

print(f"Technical Architect: {technical_response}")
print("============================================================")
print("============================================================")

# ステップ 4：市場の洞察を集める
market_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "market_researcher",
        "content": "AIカスタマーサービスプラットフォームの市場環境はどのようなものでしょうか？競合他社は？需要は？"
    }
)

print(f"Market Researcher: {market_response}")
print("============================================================")
print("============================================================")
# Manage the network
mesh_agent("コンテンツ決定委員会は終了次第閉鎖する")

### 階層型トポロジの作成 - 組織構造

最後に、明確な報告ラインを持つ組織構造を模倣した階層型トポロジを作成しましょう。

階層型構造を持つコンサルティング会社のエージェントを作成します。チームには、マネージングパートナー、戦略責任者、オペレーション責任者、戦略アナリスト、オペレーションアナリスト、データスペシャリストがいます。アーキテクチャは以下のようになります。

<div style="text-align:left">
<img src="images/hierarchical_architecture.png" width="55%" />
</div>

In [ ]:
# ステップ 1: 階層型トポロジを作成する
print("Creating hierarchical topology for organizational analysis...")

hierarchical_agent = Agent(tools=[agent_graph])
hierarchy_result = hierarchical_agent.tool.agent_graph(
    action="create",
    graph_id="consulting_firm",
    topology={
        "type": "hierarchical",
        "nodes": [
            {
                "id": "managing_partner",
                "role": "executive",
                "system_prompt": "あなたは、すべてのコンサルティング業務を統括するマネージングパートナーです。各部門長にタスクを委任し、各部門長からのレポートをまとめ、クライアントにエグゼクティブレベルの推奨事項を提供します。"
            },
            {
                "id": "strategy_head",
                "role": "department_manager",
                "system_prompt": "あなたは戦略コンサルティングの責任者です。戦略分析プロジェクトを管理し、ジュニアコンサルタントと連携し、結果をマネージングパートナーに報告します。"
            },
            {
                "id": "operations_head",
                "role": "department_manager", 
                "system_prompt": "あなたはオペレーションコンサルティングの責任者です。業務効率化プロジェクトを監督し、チームを管理し、経営陣に業務に関する洞察を提供します。"
            },
            {
                "id": "strategy_analyst",
                "role": "junior_consultant",
                "system_prompt": "あなたは、市場分析、競合情報、戦略プランニングを専門とするジュニア戦略コンサルタントです。戦略責任者に報告します。"
            },
            {
                "id": "operations_analyst",
                "role": "junior_consultant",
                "system_prompt": "プロセスの最適化、効率分析、そして業務改善に重点を置くジュニアオペレーションコンサルタントとして、オペレーション責任者にレポートしていただきます。"
            },
            {
                "id": "data_specialist",
                "role": "junior_consultant",
                "system_prompt": "あなたは、定量分析、モデリング、データ主導の洞察によって戦略と運用の両方をサポートするデータ スペシャリストです。"
            }
        ],
        "edges": [
            # 部門長のマネージングパートナー
            {"from": "managing_partner", "to": "strategy_head"},
            {"from": "managing_partner", "to": "operations_head"},
            {"from": "strategy_head", "to": "managing_partner"},
            {"from": "operations_head", "to": "managing_partner"},
            
            # 部門長からアナリストまで
            {"from": "strategy_head", "to": "strategy_analyst"},
            {"from": "strategy_head", "to": "data_specialist"},
            {"from": "operations_head", "to": "operations_analyst"},
            {"from": "operations_head", "to": "data_specialist"},
            
            # アナリストから部門長まで
            {"from": "strategy_analyst", "to": "strategy_head"},
            {"from": "operations_analyst", "to": "operations_head"},
            {"from": "data_specialist", "to": "strategy_head"},
            {"from": "data_specialist", "to": "operations_head"}
        ]
    }
)

print("Hierarchical consulting firm created!")
print(f"Result: {hierarchy_result}")
print("============================================================")
print("============================================================")

# ステップ 2: マネージングパートナーに複雑な契約書を送信する
print("Sending client engagement to managing partner...")

engagement_response = hierarchical_agent.tool.agent_graph(
    action="message",
    graph_id="consulting_firm",
    message={
        "target": "managing_partner",
        "content": "新規のクライアント案件が入りました。中規模の製造会社が、業務効率の向上を図りながら海外展開を目指しています。戦略的な市場参入アドバイスと業務最適化の両方を必要としています。包括的な分析をコーディネートしていただきますようお願いいたします。"
    }
)

print(f"Managing Partner Response: {engagement_response}")
print("============================================================")
print("============================================================")

# ステップ 3: 作業が階層に沿ってどのように流れるかを確認する
print("Checking the status of our consulting firm...")

consulting_status = hierarchical_agent.tool.agent_graph(
    action="status",
    graph_id="consulting_firm"
)

print(f"Consulting Firm Status: {consulting_status}")
print("============================================================")
print("============================================================")
# Manage the network
hierarchical_agent("コンテンツコンサルティング会社が終わったら閉鎖する")

### 自然言語インターフェース

agent_graph ツールは自然言語によるインタラクションもサポートしており、マルチエージェントシステムとの対話的な連携が容易になります。クリエイティブディレクター、ライター2名、エディター1名からなるスター型トポロジーの `creative_writers` エージェントで動作を確認してみましょう。アーキテクチャは次のようになります。

<div style="text-align:left">
<img src="images/content_writer_architecture.png" width="55%" />
</div>

In [ ]:
# ステップ 1: 自然言語によるインタラクションのためのシンプルなエージェントを作成する
nl_agent = Agent(tools=[agent_graph])

print("Using natural language to create and manage agent networks...")

# ステップ 2: 自然言語を使ってチームを作る
nl_response = nl_agent(
    "クリエイティブディレクター、ライター2名、エディター1名からなる「creative_writers」というコンテンツ制作チームを作成します。クリエイティブディレクターがチーム全体を調整するスター型トポロジを採用します。"
)

print(f"Natural Language Creation: {nl_response}")
print("============================================================")
print("============================================================")

# ステップ 3: チームにタスクを与える
task_response = nl_agent(
    "creative_writers チームに、宇宙探査における AI と人間の協力に関する SF 短編小説のアイデアをブレインストーミングしてもらいます。"
)

print(f"Creative Task Response: {task_response}")
print("============================================================")
print("============================================================")

# ステップ 4: チームの進捗状況を確認する
progress_response = nl_agent(
    "creative_writers チームの状況と、彼らが取り組んでいる内容を教えてください。"
)

print(f"Team Progress: {progress_response}")
print("============================================================")
print("============================================================")

nl_agent("creative_writersのコンテンツをシャットダウンする")


## 重要なポイントとベストプラクティス

### 各トポロジーの活用場面：

**スター型トポロジー：**
- 編集監修によるコンテンツ制作
- エスカレーションパスを備えたカスタマーサービス
- 専門家との調査連携
- 集中化された意思決定プロセス

**メッシュ型トポロジー：**
- 協調的な問題解決
- 合意形成と議論
- ピアレビュープロセス
- 民主的な意思決定

**階層型トポロジー：**
- 組織構造
- 多層的なレビュープロセス
- 委任によるプロジェクト管理
- 階層化されたデータ処理

### ベストプラクティス：

1. **明確な役割定義**：各エージェントには、明確かつ明確な役割を割り当てる必要があります。
2. **適切なシステムプロンプト**：エージェントの専門知識と役職に合わせてプロンプトを調整します。
3. **適切なクリーンアップ**：不要になったネットワークは必ず停止します。
4. **ステータス監視**：定期的にネットワークの状態を確認します。デバッグ
5. **戦略的コミュニケーション**: 特定のエージェントにターゲットを絞ったメッセージングを使用する
6. **リソース管理**: アクティブなネットワークを監視してリソースの無駄を防ぐ

## まとめ

これで、Strands Agent Graph を用いたマルチエージェントシステム構築の基礎を習得できました。複雑な問題を解決するために連携する、専門性の高い AI エージェントの洗練されたネットワークを構築できます。

マルチエージェントシステムを成功させる鍵は、以下のとおりです。
- ユースケースに適したトポロジーの採用
- エージェントの役割と責任の明確な定義
- 適切なコミュニケーションパターンの確立
- リソース管理とクリーンアップの効率化

この段階を経て、研究、コンテンツ作成、意思決定、カスタマーサービスなど、実社会でのアプリケーション向けに、より高度なシステムを構築できるようになります。